In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from scipy.sparse import csr_matrix
import random as rnd
import pickle
import sys


from datetime import datetime


from sklearn import preprocessing
from sklearn.model_selection import train_test_split



In [2]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [4]:
def generate_random_array(median,std,size):
    output=[0]*size    
    for index in range(0,size//2):
        random_nr=rnd.uniform(-std,std)
        output[2*index]=median+random_nr
        output[2*index+1]=median-random_nr
    if(size % 2 != 0):
        output[size-1]=rnd.uniform(median-std,median+std)
    return output



In [5]:
traffic = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TrafficLabelling/combined.csv")
print(f'Number of Rows: {traffic.shape[0]}')
print(f'Number of Columns: {traffic.shape[1]}')
pd.set_option('display.max_colwidth', None)
traffic.head()

Number of Rows: 2660377
Number of Columns: 85


,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,...,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.5-104.16.207.165-54865-443-6,104.16.207.165,443,192.168.10.5,54865,6,7/7/2017 3:30,3,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,33,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,192.168.10.5-104.16.28.216-55054-80-6,104.16.28.216,80,192.168.10.5,55054,6,7/7/2017 3:30,109,1,1,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,1.100917e+05,18348.62385,109.0,0.0,109.0,109.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,192.168.10.5-104.16.28.216-55055-80-6,104.16.28.216,80,192.168.10.5,55055,6,7/7/2017 3:30,52,1,1,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,2.307692e+05,38461.53846,52.0,0.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,192.168.10.16-104.17.241.25-46236-443-6,104.17.241.25,443,192.168.10.16,46236,6,7/7/2017 3:30,34,1,1,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,3.529412e+05,58823.52941,34.0,0.0,34.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,31,329,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,192.168.10.5-104.19.196.102-54863-443-6,104.19.196.102,443,192.168.10.5,54863,6,7/7/2017 3:30,3,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,32,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [7]:
# Remove duplicate entries
traffic.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)

# reformat Timestamp to Unix time
traffic[' Timestamp'] = pd.to_datetime(traffic[' Timestamp'])
traffic[' Timestamp']= traffic[' Timestamp'].values.astype(np.int64) // 10 ** 9
# Remove constant columns
traffic = traffic.loc[:, traffic.apply(pd.Series.nunique) != 1]

# Remove column with NaN or Inf
traffic = traffic[~traffic.isin([np.nan, np.inf, -np.inf]).any(1)]

# remove Flow ID
traffic = traffic.drop(columns=['Flow ID'])

# reformat Label
traffic[' Label'] = traffic[' Label'].astype('category')
traffic[' Label'] = traffic[' Label'].cat.codes

# add new "is Attacker?" column
traffic[' is Attacker?'] = np.where(traffic[' Label'] == 0, 0, 1)



KeyError: ignored

In [8]:
# look at the dataset again
print(f'Number of Rows: {traffic.shape[0]}')
print(f'Number of Columns: {traffic.shape[1]}')
traffic.head()


Number of Rows: 2657447
Number of Columns: 76


,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,is Attacker?
0,104.16.207.165,443,192.168.10.5,54865,6,3,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,40,0,666666.666700,0.000000,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,9.0,6.0,0.0,40,2,12,0,0,33,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,104.16.28.216,80,192.168.10.5,55054,6,109,1,1,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,1.100917e+05,18348.62385,109.0,0.0,109.0,109.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,20,9174.311927,9174.311927,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,104.16.28.216,80,192.168.10.5,55055,6,52,1,1,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,2.307692e+05,38461.53846,52.0,0.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,20,19230.769230,19230.769230,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,104.17.241.25,443,192.168.10.16,46236,6,34,1,1,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,3.529412e+05,58823.52941,34.0,0.0,34.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,20,29411.764710,29411.764710,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,1,6,1,6,31,329,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,104.19.196.102,443,192.168.10.5,54863,6,3,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,40,0,666666.666700,0.000000,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,9.0,6.0,0.0,40,2,12,0,0,32,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [9]:
columns = list(traffic.columns)

In [10]:
# create ip address dict {ip_addr: node_id}
ips = []        # list of ip addresses.
for ip in traffic[' Source IP']:
    if ip not in ips:
        ips.append(ip)
        
for ip in traffic[' Destination IP']:
    if ip not in ips:
        ips.append(ip)
                
ips_dict = {ips[i]: i for i in range(len(ips))}   # dictionary of ip addresses { ip_addr: node_id }
print("Number of nodes: " + str(len(ips_dict)))


Number of nodes: 18598


In [11]:
# adding a group_size column to the original dataframe
traffic['group_size'] = traffic.groupby([' Source IP'])[' Destination IP'].transform(len)
print(traffic)


              Source IP   Source Port  ...  is Attacker?  group_size
0        104.16.207.165           443  ...             0          29
1         104.16.28.216            80  ...             0         114
2         104.16.28.216            80  ...             0         114
3         104.17.241.25           443  ...             0           2
4        104.19.196.102           443  ...             0         120
...                 ...           ...  ...           ...         ...
2660372   192.168.10.14         51114  ...             0      129475
2660373   192.168.10.16         24054  ...             0      114034
2660374  23.208.163.130           443  ...             0          51
2660375   192.168.10.14         51694  ...             0      129475
2660376   192.168.10.14         57949  ...             0      129475

[2657447 rows x 77 columns]


KeyError: ignored

In [ ]:
# low activity grouping
low_activities = traffic[traffic['group_size'] < 10]
low_activities = low_activities.groupby(' Source IP', as_index=False).agg(
    {
          ' Protocol': "count",
          ' Destination Port': "count", 
          ' Destination IP': "count",
          ' Source Port': "count",
          ' Flow Duration': "mean",
            ' Total Fwd Packets': "mean",
          'Flow Bytes/s': "mean",
            ' Flow Packets/s': "mean",
          ' Flow IAT Mean' : "mean",
          ' Fwd IAT Mean': "mean",
          'Fwd Packets/s': "mean",
          ' Packet Length Mean': "mean",
          ' Average Packet Size': "mean",
          'Idle Mean': "mean",
    }
)



In [ ]:
# attackers grouping
atk_activities = traffic[traffic[' is Attacker?'] == 1]
atk_activities = atk_activities.groupby(' Source IP', as_index=False).agg(
    {
          ' Protocol': "count",
          ' Destination Port': "count", 
          ' Destination IP': "count",
          ' Source Port': "count",
          ' Flow Duration': "mean",
            ' Total Fwd Packets': "mean",
          'Flow Bytes/s': "mean",
            ' Flow Packets/s': "mean",
          ' Flow IAT Mean' : "mean",
          ' Fwd IAT Mean': "mean",
          'Fwd Packets/s': "mean",
          ' Packet Length Mean': "mean",
          ' Average Packet Size': "mean",
          'Idle Mean': "mean",
    }
)

atk_activities

,Source IP,Protocol,Destination Port,Destination IP,Source Port,Flow Duration,Total Fwd Packets,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Fwd IAT Mean,Fwd Packets/s,Packet Length Mean,Average Packet Size,Idle Mean
0,172.16.0.1,552380,552380,552380,552380,2.956412e+07,4.017171,317182.121404,96192.541960,2.907503e+06,5.546394e+06,86743.368924,431.105388,472.879368,2.719980e+07
1,192.168.10.12,2,2,2,2,3.013027e+07,5.000000,4.800454,17.423337,1.799611e+06,3.198221e+06,8.711669,15.210526,16.055556,5.100000e+06
2,192.168.10.14,209,209,209,209,4.012817e+05,4.755981,47960.846025,93.667237,7.676260e+04,1.874357e+05,46.028187,89.350356,94.335594,0.000000e+00
3,192.168.10.15,366,366,366,366,4.511831e+05,4.245902,175412.000815,23297.527461,8.735663e+04,2.140238e+05,11649.187520,69.494889,74.007528,0.000000e+00
4,192.168.10.17,2,2,2,2,3.052738e+07,5.500000,5.483256,19.827287,1.630794e+06,3.360735e+06,9.913644,15.928571,16.725000,5.000000e+06
5,192.168.10.5,179,179,179,179,4.571436e+05,4.770950,47832.096111,83.180277,8.772952e+04,2.149941e+05,40.163519,90.151575,94.789075,0.000000e+00
6,192.168.10.50,3,3,3,3,5.441377e+06,1.000000,7.237529,1.206255,1.019560e+06,0.000000e+00,0.192059,6.000000,6.952381,4.107563e+06
7,192.168.10.8,307,307,307,307,9.686172e+06,100.700326,69650.965018,8435.298142,3.334879e+05,6.584214e+05,4383.372157,65.308823,69.154867,3.121271e+06
8,192.168.10.9,226,226,226,226,3.840822e+05,4.946903,154542.879795,15571.800474,7.290158e+04,1.772273e+05,7783.485788,97.139728,102.503340,0.000000e+00
9,205.174.165.73,701,701,701,701,4.127389e+01,1.000000,659510.118981,109918.353164,4.127389e+01,0.000000e+00,54959.176581,6.000000,9.000000,0.000000e+00


In [ ]:
basic_flows = traffic.groupby([' Source IP', ' Source Port', ' Destination IP', ' Destination Port', ' Protocol'], as_index=False)

basic_flows = basic_flows.agg(
        {
            ' Flow Duration': "mean",
             ' Total Fwd Packets': "mean",
            'Flow Bytes/s': "mean",
             ' Flow Packets/s': "mean",
            ' Flow IAT Mean' : "mean",
            ' Fwd IAT Mean': "mean",
            'Fwd Packets/s': "mean",
            ' Packet Length Mean': "mean",
            ' Average Packet Size': "mean",
            'Idle Mean': "mean",
            " is Attacker?": lambda x: 1 if 1 in set(x) else 0,
        }
)
attacks = basic_flows.loc[basic_flows[' Source IP'] == '192.168.10.12'][' is Attacker?']
len(set(attacks))

2

In [ ]:
activity_flows = basic_flows.groupby([' Source IP'], as_index=False)
activity_flows = activity_flows.agg(
        {
            ' Protocol': "count",
            ' Destination Port': "count", 
            ' Destination IP': "count",
            ' Source Port': "count",
            ' Flow Duration': "mean",
             ' Total Fwd Packets': "mean",
            'Flow Bytes/s': "mean",
             ' Flow Packets/s': "mean",
            ' Flow IAT Mean' : "mean",
            ' Fwd IAT Mean': "mean",
            'Fwd Packets/s': "mean",
            ' Packet Length Mean': "mean",
            ' Average Packet Size': "mean",
            'Idle Mean': "mean",
            " is Attacker?": lambda x: 1 if 1 in set(x) else 0,
        }
)

atkers = activity_flows.groupby([' is Attacker?']).get_group(1)
non_atkers = activity_flows.groupby([' is Attacker?']).get_group(0)
print("Number of attackers: " + str(len(atkers)))
print("Number of non_atkers " + str(len(non_atkers)))

Number of attackers: 10
Number of non_atkers 16503


In [ ]:
# create hypergraph
connection = traffic.groupby([' Source IP', ' Destination IP'], as_index=False)
connection = connection.agg({' Flow Duration' : 'count'})
src_ip_ids = [ips_dict[ip] for ip in connection[' Source IP']]
dst_ip_ids = [ips_dict[ip] for ip in connection[' Destination IP']]
edges = list(zip(src_ip_ids, dst_ip_ids))
hypergraph = {i: edges[i] for i in range(len(edges))}

In [ ]:
# generate features
num_ips = len(ips_dict)
num_src_ips = len(activity_flows[' Source IP'])
num_noflow_ips = num_ips - num_src_ips

attribute_names = [' Source Port',
                   ' Destination IP',
            ' Destination Port',
            ' Flow Duration',
             ' Total Fwd Packets',
             'Flow Bytes/s',
             ' Flow Packets/s',
            ' Flow IAT Mean',
            ' Fwd IAT Mean',
            'Fwd Packets/s',
            ' Packet Length Mean',
            ' Average Packet Size',
            ]
attr_mat = np.zeros((num_ips, len(attribute_names)))
for c, name in enumerate(attribute_names):
    attr_mat[:num_src_ips, c] = activity_flows[name]


# ## TODO: CREATE LOW ACTIVITIES PROFILE 
# create low activities profile matrices
num_low_ips = round(0.65*num_noflow_ips)   #### adjust benign/attackers ratio here
low_activities_prof = np.zeros((num_low_ips, len(attribute_names)))
for col, attr in enumerate(attribute_names):
  median = low_activities[attr].median()
  std = np.std(low_activities[attr])
  low_activities_prof[:, col] = generate_random_array(median, std, num_low_ips)

# # fill no traffic ips with low acitivities profile
attr_mat[num_src_ips: num_src_ips+num_low_ips] = low_activities_prof

# CREATE ATTACKER PROFILE MATRIX
num_atk_ips = num_noflow_ips-num_low_ips
atker_prof = np.zeros((num_atk_ips, len(attribute_names)))
for col, attr in enumerate(attribute_names):
  median = atk_activities[attr].median()
  std = np.std(atk_activities[attr])
  atker_prof[:, col] = generate_random_array(median, std, num_atk_ips)

# fill no traffic ips with attackers acitivities profile
attr_mat[num_src_ips+num_low_ips:] = atker_prof


# normalise
features = preprocessing.normalize(attr_mat)
features = csr_matrix(features)



In [ ]:
# generate labels

labels = []
labels[:num_src_ips] = activity_flows[' is Attacker?']
labels[num_src_ips:num_src_ips+num_low_ips] = [0 for i in range(num_low_ips)]
labels[num_src_ips+num_low_ips:] = [ 1 for i in range(num_atk_ips)]

non_specified_count = labels.count(2)
non_atker_count = labels.count(0)
atker_count = labels.count(1)

print("Number of non-attacker:", non_atker_count)
print("Number of attacker:", atker_count)
print("Number of not specified:", non_specified_count)



Number of non-attacker: 17858
Number of attacker: 740
Number of not specified: 0


In [ ]:
# train/test split

splits = {}
attackers_ids = list(atkers.index)
V = len(ips)
node_sorted = list(range(V))
rnd.shuffle(node_sorted)
train_size = round(0.95*V)
test_size = V - train_size
train_set, test_set = train_test_split(node_sorted, train_size=train_size, test_size=test_size)



splits['train'] = train_set
splits['test'] = test_set

# for train_id in train_set:
#   if train_id in attackers_ids:
#     print("train:", train_id)

# for idx, test_id in enumerate(test_set):
#     if test_id in attackers_ids:
#       print("test:", idx, test_id)




In [ ]:
train_atk_count = 0
for idx, train_id in enumerate(train_set):
    if labels[train_id] == 1:
      train_atk_count += 1

test_atk_count = 0
for idx, test_id in enumerate(test_set):
    if labels[test_id] == 1:
      test_atk_count += 1

print("Number of atker in train set: ", train_atk_count)
print("Number of atker in test set: ", test_atk_count)

Number of atker in train set:  702
Number of atker in test set:  38


In [ ]:
# generate files
with open("hypergraph.pickle", "wb") as hypergraph_out:
    pickle.dump(hypergraph, hypergraph_out)
    hypergraph_out.close()
with open("features.pickle", "wb") as features_out:
    pickle.dump(features, features_out)
    features_out.close()
with open("labels.pickle", "wb") as labels_out:
    pickle.dump(labels, labels_out)
    labels_out.close()
with open("1.pickle", "wb") as splits_out:
    pickle.dump(splits, splits_out)
    splits_out.close()

In [ ]:
# TODO: generate ip_attr.csv file
# feature_names = [' Source Port',
#                    ' Destination IP',
#             ' Destination Port',
#             ' Flow Duration',
#              ' Total Fwd Packets',
#              'Flow Bytes/s',
#              ' Flow Packets/s',
#             ' Flow IAT Mean',
#             ' Fwd IAT Mean',
#             'Fwd Packets/s',
#             ' Packet Length Mean',
#             ' Average Packet Size',
#             'Label']

# header = ','.join(feature_names)
# print(header)
# ip_features = np.zeros((num_ips, len(attribute_names) + 1))
# print(ip_features.shape)
# ip_features = attr_mat
# ip_features[:,-1] = labels
# ip_features

In [ ]:
np.savetxt('host_data.csv', ip_features, delimiter=',', header=header)